# **Cleaning Data in Python**

## 1 Common Data Problems

### Cleaning Strings

* `pd.Series.str.strip("symbol")`
  Elimina caracteres al inicio y final de cada string en una Serie.

  * Por defecto elimina espacios en blanco.
---

### Changing Data Types

* `pd.Series.astype(dtype)`
  Convierte una Serie a un tipo de dato específico.

    ```python
    df["col"] = df["col"].astype(int)
    ```

* **Categorical type**
  Se usa para variables que toman un conjunto finito de valores.
  Ventajas:

  * Reduce el consumo de memoria.
  * Mejora el rendimiento en operaciones de `groupby` y `merge`.
  * Facilita la ordenación de categorías (ej. niveles de satisfacción: *low < medium < high*).

    ```python
    df["gender"] = df["gender"].astype("category")
    ```

---

### Working with Dates

* `pd.to_datetime(pd.Series)`
  Convierte strings en objetos de tipo `datetime64`.
  Posteriormente se pueden extraer componentes con `.dt`.

    ```python
    df["date"] = pd.to_datetime(df["date"])
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    ```

* `.dt.date`
  Extrae solo la parte de fecha (`YYYY-MM-DD`) sin la hora.

* `pd.Timestamp.today()`
  Devuelve la fecha y hora actual.
  Con `.normalize()` se obtiene solo la fecha sin hora.
  Útil para calcular diferencias de tiempo (ej. edad, antigüedad de un registro).

  ```python
  today = pd.Timestamp.today().normalize()
  df["days_since"] = today - df["date"]
  ```

---

### Handling Duplicates

* `.duplicated(subset=None, keep='first')`
  Devuelve una Serie booleana indicando si una fila es un duplicado.

  * **subset** -> columnas a considerar (por defecto todas).
  * **keep** ->

    * `'first'`: marca como duplicados todos excepto la primera aparición.
    * `'last'`: marca como duplicados todos excepto la última aparición.
    * `False`: marca todos los duplicados como `True`.

  ```python
  df[df.duplicated(subset=["id"], keep=False)]
  ```

* `.drop_duplicates(subset=None, keep='first', inplace=False)`
  Elimina las filas duplicadas según el criterio de `subset` y `keep`.


  ```python
  df.drop_duplicates(subset=["id"], keep="last", inplace=True)
  ```

* Usar `.groupby()` + `.agg()` para tratar duplicados
  En lugar de eliminar duplicados, a veces se necesita **consolidarlos** (ej. tomar el promedio, la suma o el valor más reciente).

  ```python
  df = df.groupby("id").agg({
      "score": "mean",
      "date": "max"
  }).reset_index()
  ```

---


## 2. Text and Categorical Data

### Cleaning and Transforming Strings

* `pd.Series.str.upper()`
  Convierte todos los valores de texto a mayúsculas.

* `pd.Series.str.lower()`
  Convierte todos los valores de texto a minúsculas.

* `pd.Series.str.strip()`
  Elimina espacios en blanco al inicio y final de cada string.
  También puede recibir caracteres específicos, por ejemplo:

  ```python
  df["col"] = df["col"].str.strip("$")
  ```

---

### Membership Constraints

Sirve para validar que las categorías en una columna coincidan con un conjunto esperado de valores.

Ejemplo: verificar si los valores de `cat_col` pertenecen a las categorías definidas en `categories['cat_col']`.

```python
# Encontrar categorías inconsistentes
inconsistent_categories = set(data["cat_col"]).difference(categories["cat_col"])

# Filtrar filas con categorías inconsistentes
inconsistent_rows = data["cat_col"].isin(inconsistent_categories)
inconsistent_data = data[inconsistent_rows]

# Eliminar filas inconsistentes y mantener solo las válidas
consistent_data = data[~inconsistent_rows]  # "~" es el operador "not"
```

---

### Creating Categorical Variables

* **A partir de valores continuos**

`pd.qcut` divide los datos en **cuantiles** (ej. cuartiles, deciles).

```python
df["income_group"] = pd.qcut(df["income"], q=4, labels=["Q1","Q2","Q3","Q4"])
```

`pd.cut` divide según **rangos definidos manualmente**.

```python
bins = [0, 18, 35, 60, 100]
labels = ["young", "adult", "middle", "senior"]
df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels)
```

---

### Collapsing Categories

Consiste en **agrupar o renombrar categorías** para simplificar el análisis.
Se puede lograr usando `map()` o `replace()` con diccionarios.

```python
# Ejemplo: colapsar tipos de trabajo en 2 categorías
job_map = {
    "teacher": "education",
    "professor": "education",
    "engineer": "technical",
    "developer": "technical"
}
df["job_group"] = df["job"].map(job_map)
```

---

## 3 Advanced data problems

* date formatting:
    * `pd.to_datetime(DataFrame['date_col'])`: retorna fechas segun el formato base, puede retornar erroes
    * `pd.to_datetime(DataFrame['date_col'], errors='coerce)`: las fechas con errores irreconocibles pasan a ser `NaT`
    * `pd.to_datetime(DataFrame['date_col']).dt.strftime('format')`: convertir dates a un formato dado